# imports

In [1]:
from bs4 import BeautifulSoup
from datetime import date
from requests import get
import pandas as pd
today = date.today()

# crawler

## defining url

In [2]:
url = 'https://www.pararius.com/apartments/nederland/page-'
response = get(url+'1')
html_soup = BeautifulSoup(response.text, 'html.parser')

## get numbers to define iteractions

In [3]:
num_pages = html_soup.find_all('a',{"class":"pagination__link"})[4].text
num_pages = int(num_pages)

num_rents = html_soup.find_all('div',{"class":"pagination__summary"})[0].text
num_rents = int(num_rents.split()[-2])

In [4]:
print(url+'1')
print(num_pages)
print(num_rents)

https://www.pararius.com/apartments/nederland/page-1
184
5516


## get total html soup

In [5]:
houses = []
for count in range(1, num_pages+1):
    try:
        response = get(url + str(count))
        html_soup = BeautifulSoup(response.text, 'html.parser')
        house_data = html_soup.find_all('li', class_="search-list__item search-list__item--listing")
        houses.append(house_data)
    except:
        pass

## get data

In [6]:
description_=[]
price_=[]
address_=[]
street_=[]
agency_=[]
irl_=[]
image_=[]

description_2=[]

for n_pages in range (num_pages):
    n_pages_len = len(houses[n_pages])
    for n_rents in range (n_pages_len):
        num = (houses[n_pages][n_rents])
        list_li = num.find_all('li',{"class":"illustrated-features__item"})
        description_1={}
        
        try:
            p5_value = num.find_all('span',{"class":"listing-label listing-label--under-option"})[0].text
        except:
            p5_value = None
        
        for i in range(len(list_li)):
            a = (list_li[i]["class"][1][28:])
            b = (list_li[i].text)
            description_1.update({
                a:b,
                'status':p5_value,
            })
        description_2.append(description_1)
        
        
        price = num.find_all('div',{"class":"listing-search-item__price"})[0].text
        price_.append(price)
                
        address = num.find('div',{"class":"listing-search-item__location"}).text
        address_.append(address)

        street = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0].text
        street_.append(street)

        irl = num.find_all('a',{"class":"listing-search-item__link listing-search-item__link--title"})[0]['href']
        irl_.append(irl)
        
        image = num.find_all('img')[0]['src']
        image_.append(image)

        agency = num.find_all('a', href=True)[2].text
        agency_.append(agency)

# Into DataFrame

In [7]:
df_pararius_1 = pd.DataFrame({
    'price':price_,
    'address':address_,
    'street':street_,
    'agency':agency_,
    'irl':irl_,
    'image':image_,
})
df_pararius_2 = pd.DataFrame(description_2)

## Unify dataframes

In [8]:
df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)

<ipython-input-8-22791b88bcc6>:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  df_pararius = pd.concat([df_pararius_1, df_pararius_2],1)


# fixing text

In [9]:
df_pararius['surface-area']        = df_pararius['surface-area'].str.replace("\D","",regex=True)
df_pararius['number-of-rooms']     = df_pararius['number-of-rooms'].str.replace("\D","",regex=True)
df_pararius['garden-surface-area'] = df_pararius['garden-surface-area'].str.replace("\D","",regex=True)
df_pararius['plot-size']           = df_pararius['plot-size'].str.replace("\D","",regex=True)

In [10]:
df_pararius = df_pararius.dropna(axis=1, how='all')
df_pararius['price'] = df_pararius['price'].str.replace("\D","",regex=True)
df_pararius['address'] = df_pararius['address'].str.replace('\n|new|  ',"",regex=True)
df_pararius['postcode'] = df_pararius['address'].str.replace("\s","",regex=True).str[0:6]
df_pararius['status'] = df_pararius['status'].str.replace('\n',"",regex=True)
df_pararius['date'] = str(today)

# save data

In [11]:
today_csv = ("df_pararius_"+ str(today)+'.csv')
df_pararius.to_csv(f'data/temp/{today_csv}')

In [12]:
df_pararius

,price,address,street,agency,irl,image,surface-area,status,number-of-rooms,interior,construction-period,plot-size,garden-surface-area,postcode,date
0,1550,1055 ND Amsterdam (Landlust),Apartment Admiraal De Ruijterweg,Hamilton Vastgoed,/apartment-for-rent/amsterdam/1e27decc/admiraa...,https://casco-media-prod.global.ssl.fastly.net...,60,None,3,Upholstered,NaN,NaN,NaN,1055ND,2021-10-12
1,1600,1055 ND Amsterdam (Landlust),Apartment Admiraal De Ruijterweg,Hamilton Vastgoed,/apartment-for-rent/amsterdam/fd962ef8/admiraa...,https://casco-media-prod.global.ssl.fastly.net...,65,None,3,Upholstered,NaN,NaN,NaN,1055ND,2021-10-12
2,950,2013 ZB Haarlem (Leidsebuurt-oost),Apartment Jan Nieuwenhuijzenstraat,Haarlem Real Estate,/apartment-for-rent/haarlem/17b48ca1/jan-nieuw...,https://media.pararius.nl/image/PR0001659000/P...,55,None,2,Upholstered,NaN,NaN,NaN,2013ZB,2021-10-12
3,460,3565 BA Utrecht (Bedrijventerrein en omgeving),Studio Ontariodreef,Forest Property Management,/studio-for-rent/utrecht/1c3b229d/ontariodreef,https://media.pararius.nl/image/PR0001659000/P...,16,None,1,Upholstered,NaN,NaN,NaN,3565BA,2021-10-12
4,1325,2312 GW Leiden (Marewijk),Apartment Legewerfsteeg,RE/MAX Makelaarsgilde Leiden,/apartment-for-rent/leiden/0289e309/legewerfsteeg,https://media.pararius.nl/image/PR0001659000/P...,85,None,3,Upholstered,NaN,NaN,NaN,2312GW,2021-10-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5881,1500,1505 TJ Zaandam (Bomenbuurt),House Lijsterbesstraat,Uw Partner in Onroerende Zaken,/house-for-rent/zaandam/1b23bcc5/lijsterbesstraat,https://casco-media-prod.global.ssl.fastly.net...,86,Under option,4,Upholstered,NaN,NaN,NaN,1505TJ,2021-10-12
5882,3450,3045 LJ Rotterdam (Zestienhoven),House Overschiese Kleiweg,BenHousing,/house-for-rent/rotterdam/e5d0ae4e/overschiese...,https://casco-media-prod.global.ssl.fastly.net...,294,Under option,6,Upholstered or furnished,NaN,NaN,NaN,3045LJ,2021-10-12
5883,960,3762 GK Soest (Klaarwater),Apartment Wiardi Beckmanstraat,Regio Vastgoedmakelaars,/apartment-for-rent/soest/2e33b708/wiardi-beck...,https://casco-media-prod.global.ssl.fastly.net...,90,Under option,4,Shell,NaN,NaN,NaN,3762GK,2021-10-12
5884,1350,1567 LD Assendelft (Parkrijk),Apartment Wisselwachter,Uw Partner in Onroerende Zaken,/apartment-for-rent/assendelft/829433fa/wissel...,https://casco-media-prod.global.ssl.fastly.net...,76,Under option,3,Upholstered,NaN,NaN,NaN,1567LD,2021-10-12
